In [270]:
import numpy as np
import pandas as pd
import json
import re

In [271]:
file_path = "../scraper/scraped_data/data_selenium.json"
# Load JSON file
with open(file_path) as file:
    data = json.load(file)

In [272]:
remove = ['menu', 'home', 'about us', 'contributions', 'services', 'downloads', 'gallery', 'news & events', 
          'donate us', 'vacancy', 'faqs', 'contact us', 'sitemap', 'shrama vasana fund']

In [273]:
# Convert to pandas dataframe
df = pd.DataFrame(data)
df.head()

,url,title,texts,images,pdf_links,pdf_extracted,image_extracted
0,https://www.svf.gov.lk/index.php?lang=en,Shrama Vasana Fund - Home,"[Menu, About Us, Contributions, Services, Down...","[https://www.svf.gov.lk/images/homeicon.png, h...",[],{},{'https://www.svf.gov.lk/images/homeicon.png':...
1,https://www.svf.gov.lk/index.php?option=com_co...,Shrama Vasana Fund - Overview,"[Menu, About Us, Contributions, Services, Down...","[https://www.svf.gov.lk/images/homeicon.png, h...",[],{},{'https://www.svf.gov.lk/images/homeicon.png':...
2,https://www.svf.gov.lk/index.php?option=com_co...,Shrama Vasana Fund - Contributions,"[Menu, About Us, Contributions, Services, Down...","[https://www.svf.gov.lk/images/homeicon.png, h...",[],{},{'https://www.svf.gov.lk/images/homeicon.png':...
3,https://www.svf.gov.lk/index.php?option=com_co...,Shrama Vasana Fund - Services,"[Menu, About Us, Contributions, Services, Down...","[https://www.svf.gov.lk/images/homeicon.png, h...",[],{},{'https://www.svf.gov.lk/images/homeicon.png':...
4,https://www.svf.gov.lk/index.php?option=com_co...,Shrama Vasana Fund - Downloads,"[Menu, About Us, Contributions, Services, Down...","[https://www.svf.gov.lk/images/homeicon.png, h...",[https://www.svf.gov.lk/images/pdfs/act_en.pdf...,{'https://www.svf.gov.lk/images/pdfs/act_en.pd...,{'https://www.svf.gov.lk/images/homeicon.png':...


In [274]:
text_list_sample = df.loc[0]['texts']
text_list_sample

['Menu',
 'About Us',
 'Contributions',
 'Services',
 'Downloads',
 'Gallery',
 'News & Events',
 'Donate Us',
 'Vacancy',
 'FAQs',
 'Contact Us',
 'Sitemap',
 'About Us',
 'Overview',
 'Our Team',
 'Organisation Structure',
 'Gallery',
 'Image Gallery',
 'Video Gallery',
 'Contact Us',
 'Inquiry',
 'Contact Details',
 'සිංහල',
 'தமிழ்',
 'About Us',
 'Overview',
 'Our Team',
 'Organisation Structure',
 'Contributions',
 'Services',
 'Downloads',
 'Gallery',
 'Image Gallery',
 'Video Gallery',
 'News & Events',
 'Donate Us',
 'Vacancy',
 'FAQs',
 'Contact Us',
 'Inquiry',
 'Contact Details',
 'Sitemap',
 'Health Clinics & Eye Clinics',
 'Eye Clinic',
 'Empowerment',
 'News & Events',
 'New',
 'Vacancies extended till 2024.04.26',
 'New',
 'Vacancies extended till 2024.04.26',
 'New',
 'Vacancies extended till 2024.04.26',
 'New',
 'Vacancies extended till 2024.04.26',
 'New',
 'Vacancies extended till 2024.04.26',
 'New',
 'Vacancies extended till 2024.04.26',
 'New',
 'Vacancies exten

In [275]:
def remove_duplicates(text_list):
    '''Remove duplicates and words in remove list from the text list'''
    removed = []
    for phrase in text_list: 
        if phrase not in removed:
            if phrase.lower() not in remove:
                removed.append(phrase)
    return removed

print(remove_duplicates(text_list_sample))

['Overview', 'Our Team', 'Organisation Structure', 'Image Gallery', 'Video Gallery', 'Inquiry', 'Contact Details', 'සිංහල', 'தமிழ்', 'Health Clinics & Eye Clinics', 'Eye Clinic', 'Empowerment', 'New', 'Vacancies extended till 2024.04.26', 'prev', 'next', 'Our Services', 'Promotion of the', 'Welfare of the Workers', 'Providing Financial Aid', '& Other Assistance', 'Providing Workers', 'with Medical Aid', 'Temporary Aid', 'to Workers', 'Financial Assistance', '& Other Benefits', 'Presentations in Recognition', 'of Excellent Services', 'Need Our', 'Support?', 'Your', 'Contribution', 'is Appreciated', 'Latest Lottery Results', 'Draw: 416', 'Date: Sunday,', 'September 08, 2024', 'More Results - Lucky 7', 'H', '6', '4', '0', '3', '5', '7', 'View all', 'VIDEO ARCHIVES', 'Click here', 'Related Links', 'National', 'Lotteries Board', 'Law Commission', 'of Sri Lanka', 'NILS - National', 'Institute of', 'Labour Studies', 'National Institute of', 'Occupational Safety', 'and Health', 'Ministry of La

In [276]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [277]:
stop_words = set(stopwords.words('english'))

In [278]:
def process_texts(text_list): 
    '''
    Does not include stemming and lemmatization, simply cleans the text data
    '''
    processed = []

    for phrase in text_list:
        phrase = phrase.strip().lower() # remove spaces and convert to lowercase 
        phrase = re.sub(r'[^A-Za-z0-9\s]', '', phrase) # remove special characters and punctuations from text

        processed.append(phrase) 

    return processed

In [279]:
print(process_texts(text_list_sample))

['menu', 'about us', 'contributions', 'services', 'downloads', 'gallery', 'news  events', 'donate us', 'vacancy', 'faqs', 'contact us', 'sitemap', 'about us', 'overview', 'our team', 'organisation structure', 'gallery', 'image gallery', 'video gallery', 'contact us', 'inquiry', 'contact details', '', '', 'about us', 'overview', 'our team', 'organisation structure', 'contributions', 'services', 'downloads', 'gallery', 'image gallery', 'video gallery', 'news  events', 'donate us', 'vacancy', 'faqs', 'contact us', 'inquiry', 'contact details', 'sitemap', 'health clinics  eye clinics', 'eye clinic', 'empowerment', 'news  events', 'new', 'vacancies extended till 20240426', 'new', 'vacancies extended till 20240426', 'new', 'vacancies extended till 20240426', 'new', 'vacancies extended till 20240426', 'new', 'vacancies extended till 20240426', 'new', 'vacancies extended till 20240426', 'new', 'vacancies extended till 20240426', 'prev', 'next', 'our services', 'promotion of the', 'welfare of t

In [280]:
def transform_texts(text_list): 
    '''
    Include stemming and lemmatization
    '''
    processed = []

    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    for phrase in text_list:
        phrase = phrase.strip().lower() # remove spaces and convert to lowercase 
        phrase = re.sub(r'[^A-Za-z0-9\s]', '', phrase) # remove special characters and punctuations from text
        tokens = word_tokenize(phrase) # tokenise text
        tokens = [word for word in tokens if word not in stop_words] # remove stop words
        
        #stemming
        stemming = [stemmer.stem(token) for token in tokens]

        #lemmatization
        lem = [lemmatizer.lemmatize(token) for token in stemming]

        processed.append(lem) 

    return processed

In [281]:
transform_texts(text_list_sample)

[['menu'],
 ['u'],
 ['contribut'],
 ['servic'],
 ['download'],
 ['galleri'],
 ['news', 'event'],
 ['donat', 'u'],
 ['vacanc'],
 ['faq'],
 ['contact', 'u'],
 ['sitemap'],
 ['u'],
 ['overview'],
 ['team'],
 ['organis', 'structur'],
 ['galleri'],
 ['imag', 'galleri'],
 ['video', 'galleri'],
 ['contact', 'u'],
 ['inquiri'],
 ['contact', 'detail'],
 [],
 [],
 ['u'],
 ['overview'],
 ['team'],
 ['organis', 'structur'],
 ['contribut'],
 ['servic'],
 ['download'],
 ['galleri'],
 ['imag', 'galleri'],
 ['video', 'galleri'],
 ['news', 'event'],
 ['donat', 'u'],
 ['vacanc'],
 ['faq'],
 ['contact', 'u'],
 ['inquiri'],
 ['contact', 'detail'],
 ['sitemap'],
 ['health', 'clinic', 'eye', 'clinic'],
 ['eye', 'clinic'],
 ['empower'],
 ['news', 'event'],
 ['new'],
 ['vacanc', 'extend', 'till', '20240426'],
 ['new'],
 ['vacanc', 'extend', 'till', '20240426'],
 ['new'],
 ['vacanc', 'extend', 'till', '20240426'],
 ['new'],
 ['vacanc', 'extend', 'till', '20240426'],
 ['new'],
 ['vacanc', 'extend', 'till', '202

In [282]:
def tokenise_text_list(text_list): 
    '''Tokenise text to feed into lda_model'''
    processed = []

    for phrase in text_list:
        phrase = phrase.strip().lower() # remove spaces and convert to lowercase 
        phrase = re.sub(r'[^A-Za-z0-9\s]', '', phrase) # remove special characters and punctuations from text
        tokens = word_tokenize(phrase) # tokenise text
        
        tokens = [word for word in tokens if word not in stop_words] # remove stop words
        processed.append(tokens) 

    return processed


In [283]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel

In [284]:
def topic_modelling(tokenised_text_list):
    topic_weights = [] # store topics and weights in a list [(topics, weights)]
    texts = tokenise_text_list(tokenised_text_list)
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]  
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=20)
    topics = lda_model.print_topics(num_words=1)

    for index, topic in topics:
        weight_str, topic_str = topic.split("*")
        topic = topic_str.replace('"', '')
        weight = float(weight_str)
        topic_weights.append((topic, weight))
    return topic_weights


In [285]:
topic_modelling(text_list_sample)

[('new', 0.071),
 ('us', 0.081),
 ('services', 0.054),
 ('gallery', 0.068),
 ('national', 0.041)]

In [286]:
def count_words(phrases_list): 
    word_count = 0 
    for phrase in phrases_list: 
        word_count += len(phrase.split())
    return word_count

In [287]:
# for i in range(len(data)): #loop through the data 
#     for key, value in data[i].items():
#         print(key, value)